In [ ]:
import tensorflow as tf

## Graph
---------------

In [ ]:
a = tf.constant([1., 2.], name='a')
b = tf.constant([2., 3.], name='b')
result = a + b

In [ ]:
print(a.graph is tf.get_default_graph())

In [ ]:
g1 = tf.Graph()
with g1.as_default():
    v = tf.get_variable('v', initializer=tf.zeros_initializer(), shape=[1])

In [ ]:
g2 = tf.Graph()
with g2.as_default():
    v = tf.get_variable('v', initializer=tf.ones_initializer(), shape=[1])

In [ ]:
with tf.Session(graph=g1) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope('', reuse=True):
        print(sess.run(tf.get_variable('v')))

In [ ]:
with tf.Session(graph=g2) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope('', reuse=True):
        print(sess.run(tf.get_variable('v')))

In [ ]:
g = tf.Graph()
with g.device('/gpu:0'):
    result = a + b

## Tensor
-------------

In [ ]:
a = tf.constant([1., 2.], name='a')
b = tf.constant([2., 3.], name='b')
result = tf.add(a, b, name='add')
print(result)

In [ ]:
# dtype error
a = tf.constant([1, 2], name='a')
b = tf.constant([2., 3.], name='b')
result = tf.add(a, b, name='add')
print(result)

In [ ]:
a = tf.constant([1, 2], name='a', dtype=tf.float32)
b = tf.constant([2., 3.], name='b')
result = tf.add(a, b, name='add')
print(result)

## Session
--------------

In [ ]:
sess = tf.Session()
print(sess.run(result))
sess.close()

In [ ]:
with tf.Session() as sess:
    print(sess.run(result))

In [ ]:
sess = tf.Session()
with sess.as_default():
    print(result.eval())

In [ ]:
sess = tf.InteractiveSession()
print(result.eval())
sess.close()

In [ ]:
config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)
sess = tf.InteractiveSession(config=config)
print(result.eval())
sess.close()

## Forward propagation
----------------------

In [ ]:
w1 = tf.Variable(tf.random_normal([2, 3], stddev=1., seed=1))
w2 = tf.Variable(tf.random_normal([3, 1], stddev=1., seed=1))

x = tf.constant([[0.7, 0.9]])

a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

In [ ]:
sess = tf.Session()
sess.run(w1.initializer)
sess.run(w2.initializer)
print(sess.run(y))
sess.close()

In [ ]:
w1 = tf.Variable(tf.random_normal([2, 3], stddev=1., seed=1))
w2 = tf.Variable(tf.random_normal([3, 1], stddev=1., seed=1))

x = tf.placeholder(dtype=tf.float32, shape=(3, 2), name='input')

a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

In [ ]:
sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)
print(sess.run(y, feed_dict={x: [[0.7, 0.9], [0.1, 0.4], [0.5, 0.8]]}))
sess.close()

## Full codes
------------------

In [ ]:
import tensorflow as tf
from numpy.random import RandomState

In [ ]:
batch_size = 128

w1 = tf.Variable(tf.random_normal([2, 3], stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal([3, 1], stddev=1, seed=1))

x = tf.placeholder(tf.float32, shape=(None, 2), name='x-input')
y_ = tf.placeholder(tf.float32, shape=(None, 1), name='y-input')

a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

cross_entropy = -tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y, 1e-10, 1.0)))
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

In [ ]:
rdm = RandomState(1)
dataset_size = 2^15

X = rdm.rand(dataset_size, 2)
Y = [[int(x1 + x2) < 1] for (x1, x2) in X]

In [ ]:
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print('Weigts before training:')
    print('w1: {0}'.format(sess.run(w1)))
    print('w2: {0}'.format(sess.run(w2)))
    
    steps = 5000
    for i in range(steps):
        start = (i * batch_size) % dataset_size
        end = min(start + batch_size, dataset_size)
        
        sess.run(train_step, feed_dict={x: X[start: end], y_: Y[start: end]})
        
        if (i+1) % 100 == 0:
            total_cross_entropy = sess.run(cross_entropy, feed_dict={x: X, y_: Y})
            print('After {0} training steps, cross entropy: {1}'.format(i + 1, total_cross_entropy))
            
    print('Weigts after training:')
    print('w1: {0}'.format(sess.run(w1)))
    print('w2: {0}'.format(sess.run(w2)))